<a href="https://colab.research.google.com/github/jaewoong-87/machine-learning/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- content-based filtering
- collabrative filtering

surprise libarary which provide various model.

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 5.2MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618263 sha256=78030e6bfc05776212de4887884afd1dddaf4943d87b8856742febc5cb9e5f9f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data, measures=['rmse','mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9368  0.9475  0.9270  0.9318  0.9329  0.9352  0.0069  
MAE (testset)     0.7382  0.7454  0.7307  0.7363  0.7368  0.7375  0.0047  
Fit time          4.97    5.05    4.98    4.99    4.97    5.00    0.03    
Test time         0.15    0.23    0.14    0.23    0.15    0.18    0.04    


{'fit_time': (4.9744203090667725,
  5.054948091506958,
  4.981919288635254,
  4.9945008754730225,
  4.973046064376831),
 'test_mae': array([0.73824812, 0.7453662 , 0.73069111, 0.73633312, 0.73675074]),
 'test_rmse': array([0.93683619, 0.94746549, 0.92700141, 0.93182452, 0.9328947 ]),
 'test_time': (0.14820337295532227,
  0.23226022720336914,
  0.14212989807128906,
  0.22806954383850098,
  0.1471257209777832)}

content based filtering

In [ ]:
import numpy as np

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

In [ ]:
raw_data[:,0] -=1
raw_data[:,1] -=1

In [ ]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1, n_movies +1)
shape

(943, 1682)

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] =1.
  adj_matrix

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity> best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector

print("best match:{}, best match id:{}".format(best_match, best_match_id))

best match:183, best match id:275


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 <1. and log2 >0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

euclidean (measure to distance)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print("best match:{}, best match id:{}".format(best_match, best_match_id))

best match:14.832396974191326, best match id:737


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 <1. and log2 >0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


cosine similarity

In [ ]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity 
      best_match_id = user_id
      best_match_vector = user_vector

print("best match:{}, best match id:{}".format(best_match, best_match_id))

best match:0.5278586163659506, best match id:915


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 <1. and log2 >0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector

print("best match:{}, best match id:{}".format(best_match, best_match_id))

best match:55.06359959174482, best match id:737


In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity 
      best_match_id = user_id
      best_match_vector = user_vector

print("best match:{}, best match id:{}".format(best_match, best_match_id))

best match:0.569065731527988, best match id:915


collaborative Filtering

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

In [ ]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9797  0.9845  0.9714  0.9753  0.9828  0.9787  0.0048  
MAE (testset)     0.7743  0.7752  0.7669  0.7723  0.7757  0.7729  0.0032  
Fit time          0.52    0.78    1.16    1.13    0.79    0.88    0.24    
Test time         7.52    9.50    8.66    6.81    4.83    7.47    1.61    


{'fit_time': (0.5157978534698486,
  0.7843937873840332,
  1.1595568656921387,
  1.1321511268615723,
  0.7880704402923584),
 'test_mae': array([0.77427824, 0.77523623, 0.76686866, 0.77232741, 0.7756895 ]),
 'test_rmse': array([0.97967366, 0.9845011 , 0.97136729, 0.97530174, 0.9827979 ]),
 'test_time': (7.521811008453369,
  9.502220153808594,
  8.663697957992554,
  6.808681011199951,
  4.834568023681641)}

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9379  0.9432  0.9308  0.9428  0.9290  0.9367  0.0059  
MAE (testset)     0.7399  0.7435  0.7301  0.7427  0.7335  0.7380  0.0053  
Fit time          14.07   16.70   16.79   14.52   10.23   14.46   2.39    
Test time         0.72    0.58    0.37    0.29    0.16    0.42    0.20    


{'fit_time': (14.068537712097168,
  16.695147275924683,
  16.791155099868774,
  14.518729448318481,
  10.226267576217651),
 'test_mae': array([0.73994668, 0.74351612, 0.73012775, 0.74272928, 0.73354277]),
 'test_rmse': array([0.93786974, 0.94322471, 0.9308391 , 0.94279079, 0.92895221]),
 'test_time': (0.7248988151550293,
  0.5773029327392578,
  0.36614346504211426,
  0.2915635108947754,
  0.15936708450317383)}

In [ ]:
model = NMF()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9648  0.9595  0.9660  0.9586  0.9675  0.9633  0.0036  
MAE (testset)     0.7545  0.7567  0.7611  0.7538  0.7618  0.7576  0.0033  
Fit time          13.95   17.04   17.12   14.48   10.30   14.58   2.50    
Test time         0.56    0.46    0.31    0.25    0.14    0.35    0.15    


{'fit_time': (13.950994729995728,
  17.039416313171387,
  17.120205402374268,
  14.48421335220337,
  10.2978196144104),
 'test_mae': array([0.75451499, 0.75669315, 0.76110747, 0.75384266, 0.76181658]),
 'test_rmse': array([0.96483906, 0.95950549, 0.96601674, 0.95861734, 0.96748284]),
 'test_time': (0.5585684776306152,
  0.46443605422973633,
  0.31482458114624023,
  0.2516944408416748,
  0.14449620246887207)}

In [ ]:
model = SVDpp()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

KeyboardInterrupt: ignored

hybrid

In [ ]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:,0] -=1
raw_data[:,1] -=1

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[              5,               3,               4, ...,
        139973405508808, 139973362844560, 139973362842184],
       [              4, 139973362844624, 139973362842248, ...,
        139973362915464, 139973400204744, 139973362917904],
       [139973362915528, 139973400120456, 139973362917968, ...,
        139973362466896, 139973362464520, 139973400311240],
       ...,
       [              5,               0,               0, ...,
                      0,               0,               0],
       [              0,               0,               0, ...,
                      0,               0,               0],
       [              0,               5,               0, ...,
                      0,               0,               0]])

In [ ]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [ ]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)
